# Mavlink Documentation and debugging
> Mavlink Documentaion and debugging with wireshark and QGC

[https://mavlink.io/en/mavgen_python/](https://mavlink.io/en/mavgen_python/)
[https://www.ardusub.com/developers/pymavlink.html](https://www.ardusub.com/developers/pymavlink.html)

https://mavlink.io/en/messages/common.html
https://mavlink.io/en/messages/common.html#MAV_TYPE



In [ ]:
#   # | default_exp mavlink.cam_base

In [ ]:
import UAV.logging
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| hide
# skip_showdoc: true to avoid running cells when rendering docs, and 
# skip_exec: true to skip this notebook when running tests. 
# this should be a raw cell 

***Mavlink Routing***
[https://ardupilot.org/dev/docs/mavlink-routing-in-ardupilot.html](https://ardupilot.org/dev/docs/mavlink-routing-in-ardupilot.html)
Each message contains a System ID and Component ID field to specify where the message came from. In addition some messages (including SET_POSITION_TARGET_GLOBAL_INT) include target_system and target_component fields to allow specifying which system/component should execute the command.
![](images/mavlink-routing.png){fig-align="center"}

[https://mavlink.io/en/guide/routing.html](https://mavlink.io/en/guide/routing.html) 
A MAVLINK network is made up of systems (vehicles, ground stations, antenna trackers, etc.), which may be composed from one or more components (autopilot, camera, servos, etc.).

Each system has a network-unique system id, and each component has a system-unique component id that can be used for addressing/routing:

The system id has a value between 1 and 255.
The default autopilot system id is usually 1. Users should allocate unique increasing id values when adding new autopilots to a network.
GCS systems and developer APIs typically use an ID at the top of the numeric range to reduce ID clashes (e.g. 255). Often their system ID is configurable to allow multi-GCS systems.
The component id is allocated by type and number from MAV_COMPONENT.
Messages can be intended for all systems, specific systems, all components in a system, or specific components within a system. The protocol defines two 8-bit fields that can (optionally) be specified in the message payload to indicate where the message should be sent/routed. If the ids are omitted or set to zero then the message is considered a broadcast (intended for all systems/components).

target_system: System that should execute the command
target_component: Component that should execute the command (requires target_system).
MAVLink components are expected to process messages that have a matching system/component id and broadcast messages. They are expected to route/resend messages that are intended for other (or all) recipients to other active channels (i.e. MAVLink systems may be connected across different transports, connected by a MAVLink system that routes the messages). Broadcast messages are forwarded to all channels that haven't seen the message. Addressed messages are resent on a new channel iff the system has previously seen a message from the target on that channel (messages are not resent if the addressee is not known or is on the original/incoming channel).

### Mavlink Camera Control via mavproxy serial port and UDP
*** First Run Mavproxy***
`mavproxy.py --master udpin:localhost:14445 --out udpout:localhost:14550`
`mavproxy.py --master=/dev/ttyACM1 --baudrate 57600 --out udpout:localhost:14445`


In [ ]:
# from UAV.mavlink.cam_client_server import CamClient, CamServer
import UAV as uav
import time
from fastcore.test import *

with uav.CamClient("udpin:localhost:14445", debug=True) as client:
    with uav.CamServer("udpout:localhost:14445", debug=True) as server:
        client.master.wait_heartbeat()
        
        client.trigger_camera(2)
        client.trigger_camera(2)


print(f"client.num_commands_sent: {client.num_commands_sent}")
print(f"server.num_commands_received: {server.num_commands_received}")
print(f"client.num_acks_received: {client.num_acks_received}")

print(f"server msgs: {server.message_cnts}")
print(f"client msgs: {client.message_cnts}")

test_eq(client.num_commands_sent, server.num_commands_received)
test_eq(client.num_acks_received, server.num_commands_received)


INFO   | uav.CamClient        | 20:18:14.232 |[   base.py:105] MainThread | Starting MAVLink connection... Mavlink version 2 = True
INFO   | uav.CamClient        | 20:18:14.234 |[   base.py:117] MainThread | Source system Set: 200, Source component: 25
INFO   | uav.CamClient        | 20:18:14.235 |[   base.py:118] MainThread | Target system Set: 1, Target component: 100
INFO   | uav.CamClient        | 20:18:14.336 |[   base.py:252] Thread-5 (listen) | Listening for MAVLink commands from system: 1...
INFO   | uav.CamServer        | 20:18:14.337 |[   base.py:105] MainThread | Starting MAVLink connection... Mavlink version 2 = True
INFO   | uav.CamServer        | 20:18:14.339 |[   base.py:117] MainThread | Source system Set: 1, Source component: 100
INFO   | uav.CamServer        | 20:18:14.340 |[   base.py:118] MainThread | Target system Set: 200, Target component: 25
INFO   | uav.CamServer        | 20:18:14.441 |[   base.py:155] Thread-6 (send_heartbeat) | Starting heartbeat
INFO   | uav

client.num_commands_sent: 2
server.num_commands_received: 2
client.num_acks_received: 2
server msgs: {'COMMAND_LONG': 2}
client msgs: {'COMMAND_ACK': 2}


Mavlink Camera Client

### Debugging
The above can be debugged with wireshark using the ***filter***
`mavlink_proto.sysid!=255 && not icmp`
`mavlink_proto.sysid!=255 && mavlink_proto.sysid!=1 && not icmp` 

![](images/wireshark_debug.png)



***Wireshark on Ubuntu***
see [Parsing MAVLink in Wireshark](https://mavlink.io/en/guide/wireshark.html)

Wireshark has implemented Privilege Separation which means that the Wireshark GUI (or the tshark CLI) can run as a normal user while the dumpcap capture utility runs as root. This can be achieved by installing dumpcap setuid root. The advantage of this solution is that while dumpcap is run as root the vast majority of Wireshark's code is run as a normal user (where it can do much less damage).
[https://wikileaks.org/ciav7p1/cms/page_16384719.html](https://wikileaks.org/ciav7p1/cms/page_16384719.html)
- Install Wireshark
`sudo apt-get install wireshark`
- Create a wireshark group
`sudo groupadd wireshark`
- Add your username to the wireshark group
`sudo usermod -a -G wireshark YOUR_USERNAME`
- Change the group ownership of the file dumpcap to wireshark
`sudo chgrp wireshark /usr/bin/dumpcap`
- Chage the mode of the file dumpcap to allow execution by the group wireshark
`sudo chmod 750 /usr/bin/dumpcap`
- Grant capabilities with setcap
`sudo setcap cap_net_raw,cap_net_admin=eip /usr/bin/dumpcap`
- Verify the change
`sudo getcap /usr/bin/dumpcap`
- Reboot
- `sudo reboot now`

***Also see ****
[https://wiki.wireshark.org/CaptureSetup/CapturePrivileges](https://wiki.wireshark.org/CaptureSetup/CapturePrivileges)



***Note*** 
The last few lines of the plugin file specify the ports to be monitored.
> `-- bind protocol dissector to port 14550 and 14580`
>>`local udp_dissector_table = DissectorTable.get("udp.port")`
`udp_dissector_table:add(14415, mavlink_proto)`
`udp_dissector_table:add(14425, mavlink_proto)`
`udp_dissector_table:add(14435, mavlink_proto)`
`udp_dissector_table:add(14445, mavlink_proto)`
`udp_dissector_table:add(14550, mavlink_proto)`
`udp_dissector_table:add(14580, mavlink_proto)`
`udp_dissector_table:add(18570, mavlink_proto)`



QGC can also be used to debug the communication

![](images/QGC_mavlink_inspect.png)

Enable mavlink fowarding in QGC to localhost:14445

![](images/QGC_mav_settings.png)

> Connection using a serial crossover cable or via pixhawk telemetry ports
![](../tutorials/images/serial_crossover.jpeg)  
Telemetry 2 port on pixhawk is connected to the USB port on the companion computer using a serial crossover cable.
1. (red) VCC +5V
2. (?) TX (OUT) +3.3V
3. (?) RX (IN) +3.3V
4. (?) CTS +3.3V
5. (?) RTS +3.3V
6. (?) GND GND`

#### Running PX4 SITL  
see [PX4 SITL Gazebo Simulation](https://docs.px4.io/main/en/sim_gazebo_gz/)

in the PX4 directory run
`make px4_sitl gz_x500`

*** Run Mavproxy***
`mavproxy.py --master udpin:localhost:14445 --out udpout:localhost:14550`
`mavproxy.py --master=/dev/ttyACM1 --baudrate 57600 --out udpout:localhost:14445`


https://github.com/mavlink/MAVSDK/issues/1803

So I managed to change OpenHD in this regard.
No idea why I had such a hard time wrapping my head around, but now it works the following:
OpenHD binds port 127.0.0.1:14551 and listens on 127.0.0.1:14550
AND
instead of using sendto() with a unbound port (which then in turn means the sender port can be anything) messages are sent with sendto() from the bound port (the same that is used for listening).

So messages from OpenHD to mavsdk go the following:
OpenHD (out) via 127:0:0:1:14551 sent to 127:0:0:0:1:14550

So when mavsdk receives the first message, the sender address::port is 127:0:0:1:14551 and mavsdk can send the messages back to 127:0:0:1:14551.

https://julianoes.com/
The ports are not symmetrical! QGC listens on local port 14550 and sends UDP packets back to wherever messages came from.

